In [81]:
import boto3
import pandas as pd
import io

client = boto3.client('sagemaker-runtime')
threshold = 0.5

In [70]:
f = pd.read_csv("./data/aviation_inflow.csv")
test_file = io.StringIO()
f.to_csv(test_file, header=None, index=None)

In [71]:
response = client.invoke_endpoint(
    EndpointName='ys-mlops-prod',
    Body=test_file.getvalue(),
    ContentType='text/csv',
    Accept='Accept')

In [72]:
#### Testing on a 30% split ####

import numpy as np

preds = response['Body'].read().decode('ascii')
preds = np.fromstring(preds[1:],sep=',')
die = 0
no_die = 0
for pred in preds:
    if pred >= threshold:
        die+=1
    else: no_die+=1
        
print(die, no_die)

4468 19149


In [79]:
#### Test inference run ####

test_file = io.StringIO()
inf = pd.DataFrame([[1, 2, 3, 4, 2, 3, 2, 3, 4],
                   [1, 2, 3, 4, 2, 3, 2, 3, 4],
                   [1, 2, 3, 4, 2, 3, 2, 3, 4],])
inf.to_csv(test_file, header=None, index=None)

response = client.invoke_endpoint(
    EndpointName='ys-mlops-prod',
    Body=test_file.getvalue(),
    ContentType='text/csv',
    Accept='Accept')

preds = response['Body'].read().decode('ascii')
preds = np.fromstring(preds[1:],sep=',')
die = 0
no_die = 0
for pred in preds:
    if pred >= threshold:
        die+=1
    else: no_die+=1
        
print(die, no_die)

3 0


In [86]:
#### Allow realtime input ####
#### Only enter the allowed integers please ####

print("Please enter the respective numbers: ")
incident_type = input("Incident Type? 1. Incident, 2. Accident: ")
country = input("Country? 0. Outside United States, 1. United States: ")
injury = input("Injury Severity? 0. Unavailable, 1. Minor, 2. Non-Fatal, 3. Siruouse, 4. Fatal: ")
amateur = input("Amateur Built? 0. No, 1. Yes: ")
engines = input("Number of engines: ")
fatal = input("Fatal Injuries: ")
serious = input("Serious Injuries: ")
minor = input("Minor Injuries: ")
uninjured = input("Uninjured: ")

test_file = io.StringIO()
inf = pd.DataFrame([[incident_type, country, injury, amateur, engines, fatal, serious, minor, uninjured]])
inf.to_csv(test_file, header=None, index=None)

response = client.invoke_endpoint(
    EndpointName='ys-mlops-prod',
    Body=test_file.getvalue(),
    ContentType='text/csv',
    Accept='Accept')

preds = response['Body'].read().decode('ascii')
print(preds)
preds = np.fromstring(preds[1:],sep=',')
print(preds)
if preds >= threshold:
    print("Destroyed")
else:
    print("Not destroyed")

Please enter the respective numbers: 


Incident Type? 1. Incident, 2. Accident:  2
Country? 0. Outside United States, 1. United States:  1
Injury Severity? 0. Unavailable, 1. Minor, 2. Non-Fatal, 3. Siruouse, 4. Fatal:  4
Amateur Built? 0. No, 1. Yes:  1
Number of engines:  2
Fatal Injuries:  15
Serious Injuries:  12
Minor Injuries:  3
Uninjured:  0


0.6471141576766968
[0.64711416]
Destroyed
